In [1]:
import pandas as pd
import numpy as np
import geopandas as gp
from geopy.geocoders import Nominatim

In [2]:
taxi_zone_df = pd.read_csv('https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv')

taxi_zone_df.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [3]:
taxi_zone_df.tail()

,LocationID,Borough,Zone,service_zone
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NaN,NaN
264,265,NaN,Outside of NYC,NaN


In [4]:
zone_data = pd.read_parquet('https://data.source.coop/cholmes/nyc-taxi-zones/taxi_zones.parquet')

In [5]:
zone_data.tail()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
258,259,0.12675030519,0.00039455249,Woodlawn/Wakefield,259,Bronx,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\xe6\x00...
259,260,0.13351415464,0.00042234533,Woodside,260,Queens,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x10\x01...
260,261,0.02712045636,0.00003434232,World Trade Center,261,Manhattan,"b""\x01\x03\x00\x00\x00\x01\x00\x00\x00F\x00\x0..."
261,262,0.04906362315,0.00012233027,Yorkville East,262,Manhattan,"b""\x01\x06\x00\x00\x00\x02\x00\x00\x00\x01\x03..."
262,263,0.03701662530,0.00006576977,Yorkville West,263,Manhattan,b'\x01\x03\x00\x00\x00\x01\x00\x00\x000\x00\x0...


In [6]:
len(zone_data)

263

In [7]:
zone_data.head(20)

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.11635745319,0.00078230679,Newark Airport,1,EWR,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\xe8\x00...
1,2,0.43346966679,0.00486634038,Jamaica Bay,2,Queens,b'\x01\x06\x00\x00\x00!\x00\x00\x00\x01\x03\x0...
2,3,0.08434110590,0.00031441416,Allerton/Pelham Gardens,3,Bronx,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00y\x00\x0...
3,4,0.04356652709,0.00011187195,Alphabet City,4,Manhattan,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00X\x00\x0...
4,5,0.09214648986,0.00049795749,Arden Heights,5,Staten Island,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\xaa\x00...
5,6,0.15049054252,0.00060646098,Arrochar/Fort Wadsworth,6,Staten Island,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x15\x01...
6,7,0.10741717112,0.00038978799,Astoria,7,Queens,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\xb6\x00...
7,8,0.02759069116,0.00002658772,Astoria Park,8,Queens,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00(\x00\x0...
8,9,0.09978409247,0.00033844380,Auburndale,9,Queens,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\xbd\x00...
9,10,0.09983947942,0.00043582382,Baisley Park,10,Queens,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x9d\x00...


In [8]:
jan_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')
feb_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet')
mar_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet')
apr_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-04.parquet')
may_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-05.parquet')
jun_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-06.parquet')
jul_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-07.parquet')
aug_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet')
sep_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-09.parquet')
oct_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet')
nov_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-11.parquet')
dec_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-12.parquet')


In [9]:
jan_df.head(20)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.00,0.5,0.00,0.00,1.0,22.70,2.5,0.00
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.50,0.5,3.75,0.00,1.0,18.75,2.5,0.00
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.50,0.5,3.00,0.00,1.0,31.30,2.5,0.00
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.50,0.5,2.00,0.00,1.0,17.00,2.5,0.00
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.50,0.5,3.20,0.00,1.0,16.10,2.5,0.00
5,1,2024-01-01 00:54:08,2024-01-01 01:26:31,1.0,4.70,1.0,N,148,141,1,29.6,3.50,0.5,6.90,0.00,1.0,41.50,2.5,0.00
6,2,2024-01-01 00:49:44,2024-01-01 01:15:47,2.0,10.82,1.0,N,138,181,1,45.7,6.00,0.5,10.00,0.00,1.0,64.95,0.0,1.75
7,1,2024-01-01 00:30:40,2024-01-01 00:58:40,0.0,3.00,1.0,N,246,231,2,25.4,3.50,0.5,0.00,0.00,1.0,30.40,2.5,0.00
8,2,2024-01-01 00:26:01,2024-01-01 00:54:12,1.0,5.44,1.0,N,161,261,2,31.0,1.00,0.5,0.00,0.00,1.0,36.00,2.5,0.00
9,2,2024-01-01 00:28:08,2024-01-01 00:29:16,1.0,0.04,1.0,N,113,113,2,3.0,1.00,0.5,0.00,0.00,1.0,8.00,2.5,0.00


In [10]:
dfs = [jan_df, feb_df, mar_df, apr_df, may_df, jun_df, jul_df, aug_df, sep_df, oct_df, nov_df, dec_df]

ny_taxi_2024_df = pd.concat(dfs)

In [11]:
print(f'{len(ny_taxi_2024_df):,.2f}')

41,169,720.00


In [12]:
display(ny_taxi_2024_df.head(10))
display(ny_taxi_2024_df.tail(10))

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.00
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.00
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.00
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.00
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.00
5,1,2024-01-01 00:54:08,2024-01-01 01:26:31,1.0,4.70,1.0,N,148,141,1,29.6,3.5,0.5,6.90,0.0,1.0,41.50,2.5,0.00
6,2,2024-01-01 00:49:44,2024-01-01 01:15:47,2.0,10.82,1.0,N,138,181,1,45.7,6.0,0.5,10.00,0.0,1.0,64.95,0.0,1.75
7,1,2024-01-01 00:30:40,2024-01-01 00:58:40,0.0,3.00,1.0,N,246,231,2,25.4,3.5,0.5,0.00,0.0,1.0,30.40,2.5,0.00
8,2,2024-01-01 00:26:01,2024-01-01 00:54:12,1.0,5.44,1.0,N,161,261,2,31.0,1.0,0.5,0.00,0.0,1.0,36.00,2.5,0.00
9,2,2024-01-01 00:28:08,2024-01-01 00:29:16,1.0,0.04,1.0,N,113,113,2,3.0,1.0,0.5,0.00,0.0,1.0,8.00,2.5,0.00


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
3668361,1,2024-12-31 23:21:36,2024-12-31 23:32:10,NaN,1.60,NaN,None,188,61,0,13.28,0.0,0.5,0.00,0.00,1.0,14.78,NaN,NaN
3668362,2,2024-12-31 23:02:30,2024-12-31 23:23:16,NaN,2.47,NaN,None,37,37,0,-0.90,0.0,0.5,0.00,0.00,1.0,0.60,NaN,NaN
3668363,2,2024-12-31 23:04:03,2024-12-31 23:14:58,NaN,1.96,NaN,None,137,79,0,-7.72,0.0,0.5,0.00,0.00,1.0,-3.72,NaN,NaN
3668364,1,2024-12-31 23:14:53,2024-12-31 23:35:13,NaN,2.90,NaN,None,145,164,0,21.94,0.0,0.5,0.00,6.94,1.0,32.88,NaN,NaN
3668365,2,2024-12-31 23:04:32,2024-12-31 23:19:51,NaN,2.49,NaN,None,37,36,0,-0.38,0.0,0.5,0.00,0.00,1.0,6.12,NaN,NaN
3668366,2,2024-12-31 23:32:00,2024-12-31 23:56:00,NaN,10.71,NaN,None,16,7,0,-4.13,0.0,0.5,0.00,0.00,1.0,-2.63,NaN,NaN
3668367,2,2024-12-31 23:05:00,2024-12-31 23:18:00,NaN,4.56,NaN,None,252,16,0,-1.68,0.0,0.5,0.00,0.00,1.0,-0.18,NaN,NaN
3668368,2,2024-12-31 23:03:16,2024-12-31 23:28:35,NaN,3.94,NaN,None,181,255,0,4.46,0.0,0.5,5.19,0.00,1.0,11.15,NaN,NaN
3668369,1,2024-12-31 23:15:33,2024-12-31 23:36:29,NaN,4.20,NaN,None,165,61,0,27.07,0.0,0.5,0.00,0.00,1.0,28.57,NaN,NaN
3668370,2,2024-12-31 23:04:36,2024-12-31 23:33:34,NaN,5.76,NaN,None,225,114,0,2.17,0.0,0.5,0.00,0.00,1.0,6.17,NaN,NaN


In [13]:
len(ny_taxi_2024_df['PULocationID'].loc[264])

12

In [14]:
len(ny_taxi_2024_df[ny_taxi_2024_df["PULocationID"] == 264].value_counts())

119824

In [15]:
len(ny_taxi_2024_df[ny_taxi_2024_df["DOLocationID"] == 264].value_counts())

182294

In [16]:
len(ny_taxi_2024_df[(ny_taxi_2024_df["PULocationID"] == 264) & (ny_taxi_2024_df["DOLocationID"] != 264)].value_counts())

49799

In [17]:
len(ny_taxi_2024_df[(ny_taxi_2024_df["DOLocationID"] == 264) & (ny_taxi_2024_df["PULocationID"] != 264)].value_counts())

112269

In [18]:
ny_taxi_2024_df['RatecodeID'].value_counts()

RatecodeID
1.0     34651027
2.0      1406870
99.0      466974
5.0       321948
3.0       129951
4.0       101642
6.0           76
Name: count, dtype: int64

In [19]:
ny_taxi_2024_df['passenger_count'].isna().sum()/len(ny_taxi_2024_df['passenger_count'])

0.09937478321445956

In [20]:
ny_taxi_2024_df['RatecodeID'].isna().sum()/len(ny_taxi_2024_df['RatecodeID'])

0.09937478321445956

In [21]:
ny_taxi_2024_df['congestion_surcharge'].isna().sum()/len(ny_taxi_2024_df['congestion_surcharge'])

0.09937478321445956

In [22]:
ny_taxi_2024_df['Airport_fee'].isna().sum()/len(ny_taxi_2024_df['Airport_fee'])

0.09937478321445956

In [23]:
ny_taxi_2024_df['store_and_fwd_flag'].isna().sum()/len(ny_taxi_2024_df['store_and_fwd_flag'])

0.09937478321445956

In [24]:
for column in ny_taxi_2024_df:

    print(ny_taxi_2024_df[column].isna().sum()/len(ny_taxi_2024_df[column]))

0.0
0.0
0.0
0.09937478321445956
0.0
0.09937478321445956
0.09937478321445956
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.09937478321445956
0.09937478321445956


In [25]:
ny_taxi_2024_df.dtypes

VendorID                          int32
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int32
DOLocationID                      int32
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
Airport_fee                     float64
dtype: object

In [26]:
ny_taxi_2024_df['store_and_fwd_flag'] = ny_taxi_2024_df['store_and_fwd_flag'].fillna('N')

In [27]:
ny_taxi_2024_df['passenger_count'] = ny_taxi_2024_df['passenger_count'].fillna(1)

In [28]:
ny_taxi_2024_df.tail(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
3668361,1,2024-12-31 23:21:36,2024-12-31 23:32:10,1.0,1.60,NaN,N,188,61,0,13.28,0.0,0.5,0.00,0.00,1.0,14.78,NaN,NaN
3668362,2,2024-12-31 23:02:30,2024-12-31 23:23:16,1.0,2.47,NaN,N,37,37,0,-0.90,0.0,0.5,0.00,0.00,1.0,0.60,NaN,NaN
3668363,2,2024-12-31 23:04:03,2024-12-31 23:14:58,1.0,1.96,NaN,N,137,79,0,-7.72,0.0,0.5,0.00,0.00,1.0,-3.72,NaN,NaN
3668364,1,2024-12-31 23:14:53,2024-12-31 23:35:13,1.0,2.90,NaN,N,145,164,0,21.94,0.0,0.5,0.00,6.94,1.0,32.88,NaN,NaN
3668365,2,2024-12-31 23:04:32,2024-12-31 23:19:51,1.0,2.49,NaN,N,37,36,0,-0.38,0.0,0.5,0.00,0.00,1.0,6.12,NaN,NaN
3668366,2,2024-12-31 23:32:00,2024-12-31 23:56:00,1.0,10.71,NaN,N,16,7,0,-4.13,0.0,0.5,0.00,0.00,1.0,-2.63,NaN,NaN
3668367,2,2024-12-31 23:05:00,2024-12-31 23:18:00,1.0,4.56,NaN,N,252,16,0,-1.68,0.0,0.5,0.00,0.00,1.0,-0.18,NaN,NaN
3668368,2,2024-12-31 23:03:16,2024-12-31 23:28:35,1.0,3.94,NaN,N,181,255,0,4.46,0.0,0.5,5.19,0.00,1.0,11.15,NaN,NaN
3668369,1,2024-12-31 23:15:33,2024-12-31 23:36:29,1.0,4.20,NaN,N,165,61,0,27.07,0.0,0.5,0.00,0.00,1.0,28.57,NaN,NaN
3668370,2,2024-12-31 23:04:36,2024-12-31 23:33:34,1.0,5.76,NaN,N,225,114,0,2.17,0.0,0.5,0.00,0.00,1.0,6.17,NaN,NaN


In [29]:
start_date = pd.to_datetime('2024-12-01 00:00:00')
end_date = pd.to_datetime('2024-12-31 23:33:34')

In [30]:
december_df = ny_taxi_2024_df.loc[(ny_taxi_2024_df['tpep_pickup_datetime'] >= start_date) & (ny_taxi_2024_df['tpep_pickup_datetime'] <= end_date)]
display(december_df.head())
display(december_df.tail())

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
3178038,2,2024-12-01 00:04:01,2024-12-01 00:15:11,1.0,1.97,1.0,N,114,170,1,12.8,1.0,0.5,3.56,0.0,1.0,21.36,2.5,0.0
3220341,2,2024-12-01 15:35:18,2024-12-01 15:56:58,1.0,2.74,1.0,N,237,142,1,20.5,0.0,0.5,4.90,0.0,1.0,29.40,2.5,0.0
3220342,2,2024-12-01 15:57:57,2024-12-01 16:12:50,1.0,2.61,1.0,N,142,263,1,16.3,0.0,0.5,4.06,0.0,1.0,24.36,2.5,0.0
3227441,2,2024-12-01 16:34:09,2024-12-01 16:41:29,1.0,1.59,1.0,N,143,48,1,10.0,0.0,0.5,0.00,0.0,1.0,14.00,2.5,0.0
3227442,2,2024-12-01 16:57:49,2024-12-01 17:15:24,1.0,2.26,1.0,N,68,224,1,17.0,0.0,0.5,5.25,0.0,1.0,26.25,2.5,0.0


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
3668366,2,2024-12-31 23:32:00,2024-12-31 23:56:00,1.0,10.71,NaN,N,16,7,0,-4.13,0.0,0.5,0.00,0.0,1.0,-2.63,NaN,NaN
3668367,2,2024-12-31 23:05:00,2024-12-31 23:18:00,1.0,4.56,NaN,N,252,16,0,-1.68,0.0,0.5,0.00,0.0,1.0,-0.18,NaN,NaN
3668368,2,2024-12-31 23:03:16,2024-12-31 23:28:35,1.0,3.94,NaN,N,181,255,0,4.46,0.0,0.5,5.19,0.0,1.0,11.15,NaN,NaN
3668369,1,2024-12-31 23:15:33,2024-12-31 23:36:29,1.0,4.20,NaN,N,165,61,0,27.07,0.0,0.5,0.00,0.0,1.0,28.57,NaN,NaN
3668370,2,2024-12-31 23:04:36,2024-12-31 23:33:34,1.0,5.76,NaN,N,225,114,0,2.17,0.0,0.5,0.00,0.0,1.0,6.17,NaN,NaN


In [31]:
for column in december_df:

    print(december_df[column].isna().sum()/len(december_df[column]))

0.0
0.0
0.0
0.0
0.0
0.08895266870822686
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.08895266870822686
0.08895266870822686


In [32]:
start_date = pd.to_datetime('2024-11-01 00:00:00')
end_date = pd.to_datetime('2024-11-30 23:33:34')

november_df = ny_taxi_2024_df.loc[(ny_taxi_2024_df['tpep_pickup_datetime'] >= start_date) & (ny_taxi_2024_df['tpep_pickup_datetime'] <= end_date)]
display(november_df.head(10))
display(november_df.tail(10))


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
301441,2,2024-11-14 18:30:00,2024-11-14 18:41:41,1.0,2.07,1.0,N,162,142,1,12.8,1.0,0.5,3.56,0.00,1.0,21.36,2.5,0.00
3335604,2,2024-11-01 05:16:31,2024-11-01 05:31:52,1.0,8.92,1.0,N,236,138,1,35.2,6.0,0.5,10.43,6.94,1.0,62.57,2.5,0.00
3336692,2,2024-11-01 06:13:35,2024-11-01 06:25:53,1.0,3.94,1.0,N,236,246,1,18.4,0.0,0.5,0.00,0.00,1.0,22.40,2.5,0.00
3336693,2,2024-11-01 06:53:13,2024-11-01 07:37:04,1.0,16.89,2.0,N,170,132,1,70.0,0.0,0.5,0.00,6.94,1.0,80.94,2.5,0.00
3348321,2,2024-11-01 09:55:21,2024-11-01 10:40:02,1.0,11.23,1.0,N,138,163,1,52.7,5.0,0.5,14.08,6.94,1.0,84.47,2.5,1.75
3353361,2,2024-11-01 10:42:07,2024-11-01 11:00:04,1.0,1.91,1.0,N,163,234,1,17.0,0.0,0.5,0.20,0.00,1.0,21.20,2.5,0.00
3383484,2,2024-11-01 15:09:41,2024-11-01 15:24:17,1.0,2.33,1.0,N,148,233,1,15.6,0.0,0.5,4.90,0.00,1.0,24.50,2.5,0.00
3383485,2,2024-11-01 15:46:26,2024-11-01 15:54:30,1.0,0.97,1.0,N,236,236,1,8.6,0.0,0.5,2.52,0.00,1.0,15.12,2.5,0.00
3383486,2,2024-11-01 15:57:05,2024-11-01 16:09:01,1.0,1.20,1.0,N,236,237,1,12.1,0.0,0.5,4.02,0.00,1.0,20.12,2.5,0.00
3390660,2,2024-11-01 16:20:45,2024-11-01 16:25:32,1.0,0.65,1.0,N,236,75,1,6.5,2.5,0.5,1.56,0.00,1.0,14.56,2.5,0.00


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
3646361,1,2024-11-30 23:28:30,2024-11-30 23:54:16,1.0,1.90,NaN,N,186,229,0,21.07,0.0,0.5,0.00,0.0,1.0,25.07,NaN,NaN
3646362,2,2024-11-30 23:17:00,2024-11-30 23:42:00,1.0,1.77,NaN,N,186,230,0,32.38,0.0,0.5,0.00,0.0,1.0,36.38,NaN,NaN
3646364,2,2024-11-30 23:11:15,2024-11-30 23:19:33,1.0,1.09,NaN,N,162,142,0,1.00,0.0,0.5,0.00,0.0,1.0,5.00,NaN,NaN
3646366,2,2024-11-30 23:31:46,2024-12-01 00:04:32,1.0,1.38,NaN,N,100,230,0,18.71,0.0,0.5,0.00,0.0,1.0,22.71,NaN,NaN
3646368,2,2024-11-30 23:21:52,2024-11-30 23:31:11,1.0,1.16,NaN,N,116,42,0,-0.50,0.0,0.5,0.00,0.0,1.0,1.00,NaN,NaN
935,2,2024-11-30 23:30:58,2024-11-30 23:52:32,1.0,3.93,1.0,N,100,238,1,24.00,1.0,0.5,5.80,0.0,1.0,34.80,2.5,0.0
2158,2,2024-11-30 22:31:24,2024-11-30 22:39:27,1.0,1.53,1.0,N,50,143,1,10.00,1.0,0.5,3.00,0.0,1.0,18.00,2.5,0.0
2159,2,2024-11-30 22:52:48,2024-11-30 23:01:19,1.0,0.92,1.0,N,48,68,1,9.30,1.0,0.5,2.86,0.0,1.0,17.16,2.5,0.0
2160,2,2024-11-30 23:03:00,2024-11-30 23:46:45,1.0,2.96,1.0,N,186,141,1,35.90,1.0,0.5,6.14,0.0,1.0,47.04,2.5,0.0
2247,2,2024-11-30 23:24:37,2024-11-30 23:51:15,1.0,3.19,1.0,N,230,114,1,24.00,1.0,0.5,2.00,0.0,1.0,31.00,2.5,0.0


In [33]:
for column in november_df:

    print(november_df[column].isna().sum()/len(november_df[column]))

0.0
0.0
0.0
0.0
0.0
0.10246253995861236
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.10246253995861236
0.10246253995861236


In [34]:
def find_airport_zones_vectorized(taxi_zone_df):
    airport_mask = taxi_zone_df['Zone'].str.contains('Airport', case=False, na=False) #case=False to ignore case, na=False to handle NaN values.
    airport_rows = taxi_zone_df[airport_mask].to_dict(orient='records')
    return airport_rows

In [35]:
find_airport_zones_vectorized(taxi_zone_df)

[{'LocationID': 1,
  'Borough': 'EWR',
  'Zone': 'Newark Airport',
  'service_zone': 'EWR'},
 {'LocationID': 132,
  'Borough': 'Queens',
  'Zone': 'JFK Airport',
  'service_zone': 'Airports'},
 {'LocationID': 138,
  'Borough': 'Queens',
  'Zone': 'LaGuardia Airport',
  'service_zone': 'Airports'}]

In [36]:
ny_taxi_2024_df['Airport_fee'].value_counts()

Airport_fee
 0.00    33765255
 1.75     3213953
-1.75       99253
 1.25          27
Name: count, dtype: int64

In [37]:
def impute_airport_fee(ny_taxi_2024_df):
    
    pu_mask = (ny_taxi_2024_df['PULocationID'] == 132) | (ny_taxi_2024_df['PULocationID'] == 138)
    fee_mask = np.isnan(ny_taxi_2024_df['Airport_fee']) | (ny_taxi_2024_df['Airport_fee'] == 1.25) | (ny_taxi_2024_df['Airport_fee'] == -1.75)
    mask = pu_mask & fee_mask
    ny_taxi_2024_df.loc[mask, 'Airport_fee'] = 1.75
    return ny_taxi_2024_df

In [38]:
impute_airport_fee(ny_taxi_2024_df)

ny_taxi_2024_df['Airport_fee'].value_counts()

Airport_fee
 0.00    33765255
 1.75     3323271
-1.75        2800
 1.25           4
Name: count, dtype: int64

In [39]:
def impute_outliers_airport_fee(ny_taxi_2024_df):
    
    pu_mask = (
        ny_taxi_2024_df['PULocationID'] != 132) | (
        ny_taxi_2024_df['PULocationID'] != 138) | (
            ny_taxi_2024_df['DOLocationID'] != 132) | (ny_taxi_2024_df['DOLocationID'] != 138)
    fee_mask = np.isnan(
        ny_taxi_2024_df['Airport_fee']) | (
        ny_taxi_2024_df['Airport_fee'] == 1.25) | (
            ny_taxi_2024_df['Airport_fee'] == -1.75)
    mask = pu_mask & fee_mask
    ny_taxi_2024_df.loc[mask, 'Airport_fee'] = 0
    return ny_taxi_2024_df

In [40]:
impute_outliers_airport_fee(ny_taxi_2024_df)

ny_taxi_2024_df['Airport_fee'].value_counts()

Airport_fee
0.00    37846449
1.75     3323271
Name: count, dtype: int64

In [41]:
ny_taxi_2024_df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')

In [42]:
for column in ny_taxi_2024_df:

    print(ny_taxi_2024_df[column].isna().sum()/len(ny_taxi_2024_df[column]))

0.0
0.0
0.0
0.0
0.0
0.09937478321445956
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.09937478321445956
0.0


In [43]:
ny_taxi_2024_df.tail(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
3668361,1,2024-12-31 23:21:36,2024-12-31 23:32:10,1.0,1.60,NaN,N,188,61,0,13.28,0.0,0.5,0.00,0.00,1.0,14.78,NaN,0.0
3668362,2,2024-12-31 23:02:30,2024-12-31 23:23:16,1.0,2.47,NaN,N,37,37,0,-0.90,0.0,0.5,0.00,0.00,1.0,0.60,NaN,0.0
3668363,2,2024-12-31 23:04:03,2024-12-31 23:14:58,1.0,1.96,NaN,N,137,79,0,-7.72,0.0,0.5,0.00,0.00,1.0,-3.72,NaN,0.0
3668364,1,2024-12-31 23:14:53,2024-12-31 23:35:13,1.0,2.90,NaN,N,145,164,0,21.94,0.0,0.5,0.00,6.94,1.0,32.88,NaN,0.0
3668365,2,2024-12-31 23:04:32,2024-12-31 23:19:51,1.0,2.49,NaN,N,37,36,0,-0.38,0.0,0.5,0.00,0.00,1.0,6.12,NaN,0.0
3668366,2,2024-12-31 23:32:00,2024-12-31 23:56:00,1.0,10.71,NaN,N,16,7,0,-4.13,0.0,0.5,0.00,0.00,1.0,-2.63,NaN,0.0
3668367,2,2024-12-31 23:05:00,2024-12-31 23:18:00,1.0,4.56,NaN,N,252,16,0,-1.68,0.0,0.5,0.00,0.00,1.0,-0.18,NaN,0.0
3668368,2,2024-12-31 23:03:16,2024-12-31 23:28:35,1.0,3.94,NaN,N,181,255,0,4.46,0.0,0.5,5.19,0.00,1.0,11.15,NaN,0.0
3668369,1,2024-12-31 23:15:33,2024-12-31 23:36:29,1.0,4.20,NaN,N,165,61,0,27.07,0.0,0.5,0.00,0.00,1.0,28.57,NaN,0.0
3668370,2,2024-12-31 23:04:36,2024-12-31 23:33:34,1.0,5.76,NaN,N,225,114,0,2.17,0.0,0.5,0.00,0.00,1.0,6.17,NaN,0.0


In [44]:
ny_taxi_2024_df['RatecodeID'].value_counts()

RatecodeID
1.0     34651027
2.0      1406870
99.0      466974
5.0       321948
3.0       129951
4.0       101642
6.0           76
Name: count, dtype: int64

In [48]:
ny_taxi_2024_df[ny_taxi_2024_df['RatecodeID'] == 99].head(20)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
251,1,2024-01-01 00:18:26,2024-01-01 00:52:40,1.0,0.0,99.0,N,222,147,1,52.5,0.0,0.5,0.0,6.94,1.0,60.94,0.0,0.0
1743,1,2024-01-01 00:13:45,2024-01-01 00:22:08,1.0,0.0,99.0,N,263,140,1,17.5,0.0,0.5,0.0,0.00,1.0,19.00,0.0,0.0
1922,1,2024-01-01 00:38:11,2024-01-01 01:23:26,1.0,0.0,99.0,N,41,188,1,49.5,0.0,0.5,0.0,6.94,1.0,57.94,0.0,0.0
2495,1,2024-01-01 00:43:55,2024-01-01 01:24:52,1.0,10.2,99.0,N,215,201,1,36.5,0.0,0.5,0.0,2.60,1.0,40.60,0.0,0.0
4539,1,2024-01-01 00:23:54,2024-01-01 00:50:41,1.0,0.0,99.0,N,242,50,1,38.5,0.0,0.5,0.0,0.00,1.0,40.00,0.0,0.0
4541,1,2024-01-01 00:20:51,2024-01-01 00:46:09,1.0,0.0,99.0,N,41,182,1,30.5,0.0,0.5,0.0,0.00,1.0,32.00,0.0,0.0
5231,1,2024-01-01 00:25:41,2024-01-01 00:40:55,1.0,0.0,99.0,N,83,197,1,22.5,0.0,0.5,0.0,0.00,1.0,24.00,0.0,0.0
8814,1,2024-01-01 01:20:47,2024-01-01 01:25:29,1.0,0.4,99.0,N,242,185,1,15.5,0.0,0.5,0.0,0.00,1.0,17.00,0.0,0.0
13672,1,2024-01-01 02:44:22,2024-01-01 03:41:41,1.0,17.5,99.0,N,225,208,1,53.5,0.0,0.5,0.0,6.94,1.0,61.94,0.0,0.0
21781,1,2024-01-01 04:43:31,2024-01-01 04:59:02,1.0,2.1,99.0,N,17,97,1,18.5,0.0,0.5,0.0,0.00,1.0,20.00,0.0,0.0


In [45]:
def impute_ratecodeID(ny_taxi_2024_df):
    def_mask = #parameters that effect ratecodeID based on zone
    rate_mask = (ny_taxi_2024_df['RatecodeID'] > 6) | np.isnan(ny_taxi_2024_df['RatecodeID'])
    mask = def_mask & rate_mask
    ny_taxi_2024_df.loc[mask, ['RatecodeID']] = 1

SyntaxError: invalid syntax (3867786144.py, line 2)